<img src="what_is_video_QA.png" width="600" height="600">

<img src="video_QA_model.png" width="600" height="600">

## Visual question answering model

- This model can select the correct one-word answer when asked a natural-language question about a picture.

- It works by encoding the question into a vector, encoding the image into a vector, concatenating the two, and training on top a logistic regression over some vocabulary of potential answers.

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model, Sequential

# First, let's define a vision model using a Sequential model.
# This model will encode an image into a vector.
vision_model = Sequential()
vision_model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
vision_model.add(Conv2D(64, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
vision_model.add(Conv2D(128, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
vision_model.add(Conv2D(256, (3, 3), activation='relu'))
vision_model.add(Conv2D(256, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Flatten())

# Now let's get a tensor with the output of our vision model:
image_input = Input(shape=(224, 224, 3))
encoded_image = vision_model(image_input)

# Next, let's define a language model to encode the question into a vector.
# Each question will be at most 100 word long,
# and we will index words as integers from 1 to 9999.
question_input = Input(shape=(100,), dtype='int32')
embedded_question = Embedding(input_dim=10000, output_dim=256, input_length=100)(question_input)
encoded_question = LSTM(256)(embedded_question)

# Let's concatenate the question vector and the image vector:
merged = keras.layers.concatenate([encoded_question, encoded_image])

# And let's train a logistic regression over 1000 words on top:
output = Dense(1000, activation='softmax')(merged)

# This is our final model:
vqa_model = Model(inputs=[image_input, question_input], outputs=output)

# The next stage would be training this model on actual data.

## Video question answering model

- Now that we have trained our image QA model, we can quickly turn it into a video QA model. 

- With appropriate training, you will be able to show it a short video (e.g. 100-frame human action) and ask a natural language question about the video (e.g. "what sport is the boy playing?" -> "football").

In [ ]:
from keras.layers import TimeDistributed

video_input = Input(shape=(100, 224, 224, 3))
# This is our video encoded via the previously trained vision_model (weights are reused)
encoded_frame_sequence = TimeDistributed(vision_model)(video_input)  # the output will be a sequence of vectors
encoded_video = LSTM(256)(encoded_frame_sequence)  # the output will be a vector

# This is a model-level representation of the question encoder, reusing the same weights as before:
question_encoder = Model(inputs=question_input, outputs=encoded_question)

# Let's use it to encode the question:
video_question_input = Input(shape=(100,), dtype='int32')
encoded_video_question = question_encoder(video_question_input)

# And this is our video question answering model:
merged = keras.layers.concatenate([encoded_video, encoded_video_question])
output = Dense(1000, activation='softmax')(merged)
video_qa_model = Model(inputs=[video_input, video_question_input], outputs=output)

# How to define custom cost (loss) function in Keras? How to define custom layer operation in Keras?

- Read: http://faroit.com/keras-docs/2.0.2/backend/

In [12]:
from keras import backend as K
import numpy as np

a = np.array([1, 2, 3])
b = K.sum(a, axis= -1)
print(b)
K.eval(b)
# c = K.mean(K.variable(a), axis=-1)
c = K.mean(K.constant(a), axis=-1)

K.eval(c)

Tensor("Sum_3:0", shape=(), dtype=int64)


2.0

In [6]:
import keras

y = K.variable(np.array([0, 1]))
yhat = K.variable(np.array([0.1, 0.9]))
K.eval(keras.losses.binary_crossentropy(y, yhat))

0.10536052

In [7]:
def identity_loss(y_true, y_pred):

    return K.mean(y_pred - 0 * y_true)

In [11]:
K.eval(identity_loss(y, yhat))

0.5

In [ ]:
"""
Triplet loss network example for recommenders
https://arxiv.org/pdf/1205.2618.pdf
"""

from __future__ import print_function

import numpy as np

from keras import backend as K
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, merge
from keras.optimizers import Adam

import data
import metrics


def identity_loss(y_true, y_pred):

    return K.mean(y_pred - 0 * y_true)


def bpr_triplet_loss(X):

    positive_item_latent, negative_item_latent, user_latent = X

    # BPR loss
    loss = 1.0 - K.sigmoid(
        K.sum(user_latent * positive_item_latent, axis=-1, keepdims=True) -
        K.sum(user_latent * negative_item_latent, axis=-1, keepdims=True))

    return loss


def build_model(num_users, num_items, latent_dim):

    positive_item_input = Input((1, ), name='positive_item_input')
    negative_item_input = Input((1, ), name='negative_item_input')

    # Shared embedding layer for positive and negative items
    item_embedding_layer = Embedding(
        num_items, latent_dim, name='item_embedding', input_length=1)

    user_input = Input((1, ), name='user_input')

    positive_item_embedding = Flatten()(item_embedding_layer(
        positive_item_input))
    negative_item_embedding = Flatten()(item_embedding_layer(
        negative_item_input))
    user_embedding = Flatten()(Embedding(
        num_users, latent_dim, name='user_embedding', input_length=1)(
            user_input))

    loss = merge(
        [positive_item_embedding, negative_item_embedding, user_embedding],
        mode=bpr_triplet_loss,
        name='loss',
        output_shape=(1, ))

    model = Model(
        input=[positive_item_input, negative_item_input, user_input],
        output=loss)
    model.compile(loss=identity_loss, optimizer=Adam())

    return model


if __name__ == '__main__':

    latent_dim = 100
    num_epochs = 10

    # Read data
    train, test = data.get_movielens_data()
    num_users, num_items = train.shape

    # Prepare the test triplets
    test_uid, test_pid, test_nid = data.get_triplets(test)

    model = build_model(num_users, num_items, latent_dim)

    # Print the model structure
    print(model.summary())

    # Sanity check, should be around 0.5
    print('AUC before training %s' % metrics.full_auc(model, test))

    for epoch in range(num_epochs):

        print('Epoch %s' % epoch)

        # Sample triplets from the training data
        uid, pid, nid = data.get_triplets(train)

        X = {
            'user_input': uid,
            'positive_item_input': pid,
            'negative_item_input': nid
        }

        model.fit(X,
                  np.ones(len(uid)),
                  batch_size=64,
                  nb_epoch=1,
                  verbose=0,
                  shuffle=True)

        print('AUC %s' % metrics.full_auc(model, test))
